<a href="https://colab.research.google.com/github/rubaahmedkhan/AgenticAI-Toolkit-/blob/main/Tracing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.4 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import os

from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel, ModelSettings, function_tool
from agents.run import RunConfig
from google.colab import userdata


In [ ]:
gemini_api_key = userdata.get("GEMINI_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)
model_settings = ModelSettings(
    max_tokens=50,
                                              # CHECK All PARAMETERS PICTURE IN MOBILE
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    model_settings=model_settings,
    tracing_disabled=True
)

In [ ]:
agent = Agent(
    name="helpful assistant",
    instructions="You are a helpful assistant.",
)

result = await Runner.run(agent, "hello, how are you", run_config=config)
print(result)


RunResult:
- Last agent: Agent(name="helpful assistant", ...)
- Final output (str):
    Hello! As an AI, I don't experience feelings in the same way humans do. But I'm functioning well and ready to assist you. How can I help you today?
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [ ]:
result.last_agent

Agent(name='helpful assistant', instructions='You are a helpful assistant.', prompt=None, handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [ ]:
result.output_guardrail_results

[]

In [ ]:
result.new_items

[MessageOutputItem(agent=Agent(name='helpful assistant', instructions='You are a helpful assistant.', prompt=None, handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), raw_item=ResponseOutputMessage(id='__fake_id__', content=[ResponseOutputText(annotations=[], text='Hello! How can I help you today?\n', type='output_text', logprobs=None)], role='assistant', status='completed', type='message'), type='message_output_item')]

In [ ]:
result.raw_responses

[ModelResponse(output=[ResponseOutputMessage(id='__fake_id__', content=[ResponseOutputText(annotations=[], text="Hello! As an AI, I don't experience feelings in the same way humans do, but I am functioning well and ready to assist you. How can I help you today?\n", type='output_text', logprobs=None)], role='assistant', status='completed', type='message')], usage=Usage(requests=1, input_tokens=11, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=38, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=49), response_id=None)]

# **💡 Trace aur Span ka matlab kya hai?**
Jab aap kisi AI workflow ko chalatay hain — jaise ke ek agent user se baat karta hai, tool call karta hai, LLM se jawab mangta hai — to poora process ko ek Trace kehtay hain.

Aur Trace ke andar chhoti chhoti activities ko Spans kehtay hain.

🟦 Trace – Pura Workflow ka Record
Trace represent karta hai ek poora end-to-end kaam — jaise:

"Customer Support Agent ne user ka masla hal kia"

Trace ke Properties:
workflow_name – Workflow ka naam, jaise "Code generation" ya "Customer support".

trace_id – Unique ID hoti hai har trace ki. Format: trace_<32 letters/numbers>. (Agar na dein to system khud bana deta hai.)

group_id – Optional hai. Agar aap multiple traces ko ek group mein rakhna chahtay hain (jaise ek hi user ka chat session), to yeh use hota hai.

disabled – Agar True ho, to trace record nahi hoti.

metadata – Aap kuch extra information bhi trace ke sath attach kar saktay hain (jaise user ID, country etc.)


🟨 Span – Trace ke Andar Ek Chhota Hissa
Har trace ke andar chhoti chhoti steps hoti hain, jinko Spans kehtay hain.

Jaise:

Agent call karna

LLM se response lena

Tool run karna

Data store karna

Span ke Properties:
started_at & ended_at – Start aur end ka time.

trace_id – Ye span kis trace se related hai.

parent_id – Agar yeh span kisi aur span ke andar hua, to uska ID.

span_data – Yeh batata hai span kis kaam ke liye tha:

AgentSpanData – agar agent chal raha tha

GenerationSpanData – agar LLM se jawab aaya tha

ToolSpanData – agar koi tool run hua tha


**✅ Default Tracing ka Matlab**
Jab aap OpenAI Agent SDK ka use karte hain, to SDK khud ba khud (automatically) kuch important cheezon ka trace record karta hai — aapko manually kuch likhne ki zarurat nahi hoti.


| Kaam                                               | Span Type              | Kya Trace Hota Hai?                    |
| -------------------------------------------------- | ---------------------- | -------------------------------------- |
| `Runner.run()` ya `run_sync()` ya `run_streamed()` | `trace()`              | Poora execution                        |
| Agent ka chalna                                    | `agent_span()`         | Jab agent kaam kar raha ho             |
| LLM ka jawab dena                                  | `generation_span()`    | LLM se prompt bhejna aur response lena |
| Tool ka call hona                                  | `function_span()`      | Jab koi tool use ho                    |
| Guardrail ka kaam                                  | `guardrail_span()`     | Agar guardrails lage ho                |
| Handoff (ek agent se doosray ko kaam dena)         | `handoff_span()`       | Jab agent doosray ko handoff kare      |
| Audio input (Speech to Text)                       | `transcription_span()` | User ki awaaz ko text mein badalna     |
| Audio output (Text to Speech)                      | `speech_span()`        | Text ko awaaz mein badalna             |
| Related audio spans grouping                       | `speech_group_span()`  | Sab audio spans ko group karna         |



🏷️ **Default Trace Name**
SDK trace ka default naam hota hai: "Agent trace"

Lekin agar aap chahein to trace() ya RunConfig mein naam aur properties customize kar saktay hain.




In [ ]:

from agents import Runner,RunConfig

# Custom RunConfig
run_config = RunConfig(
    trace_name="Customer Support Workflow",  # custom trace name
    trace_metadata={"user_id": "user_123", "session": "chat_456"},
    trace_group_id="conversation_group_1"
)

# Runner with the custom config
result = await Runner.run(agent=agent, input="Hello!", run_config=run_config)


TypeError: RunConfig.__init__() got an unexpected keyword argument 'trace_name'

# **Higher level traces**
Sometimes, you might want multiple calls to run() to be part of a single trace. You can do this by wrapping the entire code in a trace().

In [ ]:
from agents import Agent, Runner, trace

async def main():
    agent = Agent(name="Joke generator", instructions="Tell funny jokes.")

    with trace("Joke workflow"):
        first_result = await Runner.run(agent, "Tell me a joke",run_config=config)
        second_result = await Runner.run(agent, f"Rate this joke: {first_result.final_output}",run_config=config)
        print(f"Joke: {first_result.final_output}")
        print(f"Rating: {second_result.final_output}")

# For running the async main
if __name__ == "__main__":
    asyncio.run(main())



Joke: Why don't scientists trust atoms?

Because they make up everything!

Rating: Okay, that's a solid classic! I'd give it a **7/10**. It's punny, easily understandable, and gets a chuckle. It's not going to win any comedy awards, but it's


# **Trace Locally**

In [ ]:
import os
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, set_default_openai_api, set_default_openai_client, set_trace_processors
from agents.tracing.processor_interface import TracingProcessor
from pprint import pprint
# Custom trace processor to collect trace data locally
class LocalTraceProcessor(TracingProcessor):
    def __init__(self):
        self.traces = []
        self.spans = []

    def on_trace_start(self, trace):
        self.traces.append(trace)
        print(f"Trace started: {trace.trace_id}")

    def on_trace_end(self, trace):
        print(f"Trace ended: {trace.export()}")

    def on_span_start(self, span):
        self.spans.append(span)
        print(f"Span started: {span.span_id}")
        print(f"Span details: ")
        pprint(span.export())

    def on_span_end(self, span):
        print(f"Span ended: {span.span_id}")
        print(f"Span details:")
        pprint(span.export())

    def force_flush(self):
        print("Forcing flush of trace data")

    def shutdown(self):
        print("=======Shutting down trace processor========")
        # Print all collected trace and span data
        print("Collected Traces:")
        for trace in self.traces:
            print(trace.export())
        print("Collected Spans:")
        for span in self.spans:
            print(span.export())

BASE_URL = os.getenv("BASE_URL") or "https://generativelanguage.googleapis.com/v1beta/openai/"
API_KEY = os.getenv("GEMINI_API_KEY") or userdata.get("GEMINI_API_KEY")
MODEL_NAME = os.getenv("MODEL_NAME") or "gemini-2.0-flash"


if not BASE_URL or not API_KEY or not MODEL_NAME:
    raise ValueError("Please set BASE_URL, GEMINI_API_KEY, MODEL_NAME via env var or code.")

# Create OpenAI client
client = AsyncOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
)

# Configure the client
set_default_openai_client(client=client, use_for_tracing=True)
set_default_openai_api("chat_completions")

# Set up the custom trace processor
local_processor = LocalTraceProcessor()
set_trace_processors([local_processor])

# Example function to run an agent and collect traces
async def main():
    agent = Agent(name="Example Agent", instructions="Perform example tasks.", model=MODEL_NAME)

    with trace("Example workflow"):
        first_result = await Runner.run(agent, "Start the task")
        second_result = await Runner.run(agent, f"Rate this result: {first_result.final_output}")
        print(f"Result: {first_result.final_output}")
        print(f"Rating: {second_result.final_output}")

# Run the main function
import asyncio
asyncio.run(main())

Trace started: trace_c8db5f792bc64b3998ca730338b846e7
Span started: span_cf82ca08443d4910901e2ef8
Span details: 
{'ended_at': None,
 'error': None,
 'id': 'span_cf82ca08443d4910901e2ef8',
 'object': 'trace.span',
 'parent_id': None,
 'span_data': {'handoffs': [],
               'name': 'Example Agent',
               'output_type': 'str',
               'tools': None,
               'type': 'agent'},
 'started_at': '2025-06-26T06:14:05.530483+00:00',
 'trace_id': 'trace_c8db5f792bc64b3998ca730338b846e7'}
Span started: span_b95f6d422274426da485ac0e
Span details: 
{'ended_at': None,
 'error': None,
 'id': 'span_b95f6d422274426da485ac0e',
 'object': 'trace.span',
 'parent_id': 'span_cf82ca08443d4910901e2ef8',
 'span_data': {'input': None,
               'model': 'gemini-2.0-flash',
               'model_config': {'base_url': 'https://generativelanguage.googleapis.com/v1beta/openai/',
                                'extra_args': None,
                                'extra_body': None,
  